In [5]:
from sequential_analysis import compute_max_error_probability, compute_error_probability
import numpy as np
import sys
import numpy as np
from explicit_summaries import encode_assertion, encode_distribution, save_explicit_summaries
from analysis import Polyhedron
# Ensure the parent directory is in the path before attempting the import
try:
    import summary_generation as sg
except ModuleNotFoundError:
    # Add the parent directory to sys.path
    sys.path.insert(0, '../compiler')
    # Try the import again
    import summary_generation as sg
    import util as cutil
    del sys.path[0]

In [6]:
summary_state_file = '../f1tenth/summary_programs/base_05noise_summarized_multi_init_SUMMARIES_STATES.txt'
summary_np_file = '../f1tenth/base_05noise_summarized_multi_init_SUMMARIES.npz'

In [7]:
# This Cell creates the results for F1Tenth in Table 2 of the CAV submission.
f1_summaries_path = summary_np_file

def char_to_segment(c):
    if c in ['l', 'L']:
        return 0
    elif c in ['r', 'R']:
        return 1
    elif c in ['s', 'S']:
        return 2
    else:
        raise ValueError("Illegal char "+ c)

def track_to_segment_list(track_str):
    return [char_to_segment(c) for c in track_str]

# nb I named these all wrong- right and left need to be swapped
f1_program_names = [
    "LLLL",
    "RRRR",
    "SSSS",
    "RSSL",
    "RSSR",
]
f1_programs = [(f1_summaries_path, name, track_to_segment_list(name)) for name in f1_program_names]
print("These are your programs:")
print(f1_programs)

# Building blocks (state predicates) for the preconditions.
def f1_not_straight(s):
    return s['car_heading'] != 0
def f1_not_straightish(s):
    return s['car_heading'] not in [7, 0, 1]
def f1_not_center(s):
    return s['car_dist_s'] != 0
def f1_not_centerish(s):
    return s['car_dist_s'] not in [-1,0,1]
def f1_not_right(s):
    return s['car_dist_s'] != 2
def f1_not_both_straight_and_right(s):
    return f1_not_straight(s) or f1_not_right(s)
def f1_not_both_straightish_and_right(s):
    return f1_not_straightish(s) or f1_not_right(s)
def f1_not_on_track(s):
    return s['car_dist_s'] not in [-2,-1,0,1,2]
def f1_not_both_straight_and_center(s):
    return f1_not_straight(s) or f1_not_center(s)
def f1_not_both_straightish_and_centerish(s):
    return f1_not_straightish(s) or f1_not_centerish(s)
def f1_not_both_straightish_and_exactly_center(s):
    return f1_not_straightish(s) or f1_not_center(s)
def f1_not_both_centerish_and_exactly_straight(s):
    return f1_not_centerish(s) or f1_not_straight(s)

def f1_not_somewhat_forward(s):
    return s['car_heading'] not in [6,7,0,1,2]

for summary_path, program_name, scenario_sequence in f1_programs:
    print("--------------")
    print(program_name)
    print(f"Nominal. Pr(straightish) =1 and Pr(straight and center) > 0.9 yields max err: {compute_max_error_probability(summary_path, scenario_sequence, [f1_not_both_straight_and_center, f1_not_straightish], [0.1, 0])}")
    print(f"Center. Pr(straightish)=1 and Pr(centerish)>0.9 yields max err: {compute_max_error_probability(summary_path, scenario_sequence, [f1_not_straightish, f1_not_centerish], [0, 0.1])}")
    print(f"Right. Pr(straightish)=1 and Pr(exactly right)>0.9 yields max err: {compute_max_error_probability(summary_path, scenario_sequence, [f1_not_straightish, f1_not_right], [0, 0.1])}")


These are your programs:
[('../f1tenth/base_05noise_summarized_multi_init_SUMMARIES.npz', 'LLLL', [0, 0, 0, 0]), ('../f1tenth/base_05noise_summarized_multi_init_SUMMARIES.npz', 'RRRR', [1, 1, 1, 1]), ('../f1tenth/base_05noise_summarized_multi_init_SUMMARIES.npz', 'SSSS', [2, 2, 2, 2]), ('../f1tenth/base_05noise_summarized_multi_init_SUMMARIES.npz', 'RSSL', [1, 2, 2, 0]), ('../f1tenth/base_05noise_summarized_multi_init_SUMMARIES.npz', 'RSSR', [1, 2, 2, 1])]
--------------
LLLL
Restricted license - for non-production use only - expires 2026-11-23
Nominal. Pr(straightish) =1 and Pr(straight and center) > 0.9 yields max err: 0.3381593559175
Restricted license - for non-production use only - expires 2026-11-23
Center. Pr(straightish)=1 and Pr(centerish)>0.9 yields max err: 0.5862385127008
Restricted license - for non-production use only - expires 2026-11-23
Right. Pr(straightish)=1 and Pr(exactly right)>0.9 yields max err: 0.803848180015
--------------
RRRR
Restricted license - for non-prod

In [10]:
taxinet_summaries_path = '../taxinet/bright_dark/taxinet_5bins_joint_SUMMARIES.npz'
taxinet_pooled_summaries_path = '../taxinet/pooled/taxinet_Boeing5bins_day_and_night_mixed_summarized_multi_init_SUMMARIES.npz'
programs = [
    (taxinet_summaries_path, "bright_bright", [0,0]),
    (taxinet_summaries_path, "bright_dark", [0,1]),
    (taxinet_summaries_path, "dark_bright", [1,0]),
    (taxinet_summaries_path, "dark_dark", [1,1]),
    (taxinet_pooled_summaries_path, "pooled_pooled", [0,0])
]

# Building blocks (state predicates) for atomic assertions
def taxi_not_is_straight(s):
    return s['he'] != 0
def taxi_not_is_center(s):
    return s['cte'] != 0
def taxi_not_both_straight_and_center(s):
    return taxi_not_is_straight(s) or taxi_not_is_center(s)
def taxi_not_centerish(s):
    return s['cte'] >= 3
def taxi_not_right_and_facing_leftish(s):
    return (s['cte'], s['he']) not in [(4, 0), (4,1)]
def taxi_not_right(s):
    return s['cte'] not in [4]

In [11]:
for summary_path, program_name, scenario_sequence in programs:
    print("--------------")
    print(program_name)
    print(f"X_nominal aka Pr(straight and center) > 0.9  yields max err: {compute_max_error_probability(summary_path, scenario_sequence, [taxi_not_both_straight_and_center], [0.1])}")
    print(f"X_center aka Pr(cte >= 3) <= 0.9 yields max err: {compute_max_error_probability(summary_path, scenario_sequence, [taxi_not_centerish], [0.1])}")
    print(f"X_right aka Pr(cte = 4) > 0.9 yields max err: {compute_max_error_probability(summary_path, scenario_sequence, [taxi_not_right], [0.1])}")


--------------
bright_bright
Restricted license - for non-production use only - expires 2026-11-23
X_nominal aka Pr(straight and center) > 0.9  yields max err: 0.0301786
Restricted license - for non-production use only - expires 2026-11-23
X_center aka Pr(cte >= 3) <= 0.9 yields max err: 0.099016
Restricted license - for non-production use only - expires 2026-11-23
X_right aka Pr(cte = 4) > 0.9 yields max err: 0.21450219999999998
--------------
bright_dark
Restricted license - for non-production use only - expires 2026-11-23
X_nominal aka Pr(straight and center) > 0.9  yields max err: 0.32309490000000013
Restricted license - for non-production use only - expires 2026-11-23
X_center aka Pr(cte >= 3) <= 0.9 yields max err: 0.37114860000000016
Restricted license - for non-production use only - expires 2026-11-23
X_right aka Pr(cte = 4) > 0.9 yields max err: 0.45182370000000005
--------------
dark_bright
Restricted license - for non-production use only - expires 2026-11-23
X_nominal aka Pr